In [1]:
pip install -U "accelerate>=0.26.0"

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install einops timm

import sys, os
!git clone https://github.com/Sakib323/AI-Game-Engine.git
sys.path.append('/workspace/AI-Game-Engine') 
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
fatal: destination path 'AI-Game-Engine' already exists and is not an empty directory.


In [6]:
import os, json, shutil, logging, random, signal, time
import numpy as np
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, Subset, RandomSampler, SequentialSampler, DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments
from diffusers import DDIMScheduler
from safetensors.torch import load_file as safetensors_load
import wandb

import torch._dynamo
torch._dynamo.config.suppress_errors = True   
torch._dynamo.config.verbose = False

os.environ["TOKENIZERS_PARALLELISM"] = "false"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print(f"Using device: {device}, dtype: {dtype}")              

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded.")

print("SUCCESS: MeshDiT models imported!")

Using device: cuda, dtype: torch.float32
Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Tokenizer loaded.
SUCCESS: MeshDiT models imported!


In [4]:
!pip install kagglehub
import kagglehub

path = kagglehub.dataset_download("sakibahmed2022/meshdit-all-data-5-pt")
print("Dataset downloaded to:", path)
cache_dir = "/tmp/meshdit_cache"
os.makedirs(cache_dir, exist_ok=True)

file_names = ["all_data.pt", "all_data_1.pt", "all_data_2.pt", "all_data_3.pt", "all_data_4.pt"]
local_paths = []

for name in file_names:
    src = os.path.join(path, name)
    dst = os.path.join(cache_dir, name)
    
    if not os.path.exists(dst):
        print(f"Copying {name} → {dst}")
        shutil.copy2(src, dst)
    else:
        print(f"Already cached: {dst}")
        
    local_paths.append(dst)

print("All .pt files ready in", cache_dir)

100%|██████████| 887M/887M [00:07<00:00, 119MB/s]  

Extracting files...


100%|██████████| 44.1G/44.1G [08:09<00:00, 96.6MB/s]  

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/sakibahmed2022/meshdit-all-data-5-pt/versions/1
Copying all_data.pt → /tmp/meshdit_cache/all_data.pt
Copying all_data_1.pt → /tmp/meshdit_cache/all_data_1.pt
Copying all_data_2.pt → /tmp/meshdit_cache/all_data_2.pt
Copying all_data_3.pt → /tmp/meshdit_cache/all_data_3.pt
Copying all_data_4.pt → /tmp/meshdit_cache/all_data_4.pt
All .pt files ready in /tmp/meshdit_cache


In [5]:
print("\n--- Freeing Disk Space ---")

if os.path.exists(path):
    print(f"Deleting original Kaggle cache at: {path}")
    shutil.rmtree(path)
    print("✅ Redundant dataset deleted.")

total, used, free = shutil.disk_usage("/")
print(f"\nDISK SPACE REMAINING: {free // (2**30)} GB")
print("All .pt files ready in", cache_dir)


--- Freeing Disk Space ---
Deleting original Kaggle cache at: /root/.cache/kagglehub/datasets/sakibahmed2022/meshdit-all-data-5-pt/versions/1
✅ Redundant dataset deleted.

DISK SPACE REMAINING: 49 GB
All .pt files ready in /tmp/meshdit_cache


In [9]:
def train_model():
    import os, random, shutil
    import torch
    import torch.nn.functional as F
    from torch.utils.data import Dataset, Subset, RandomSampler, SequentialSampler, DataLoader
    from transformers import Trainer, TrainingArguments, TrainerCallback 
    from diffusers import DDIMScheduler
    from tqdm.notebook import tqdm
    import wandb

    import torch._dynamo
    torch._dynamo.config.suppress_errors = True   
    torch._dynamo.config.verbose = False

    WANDB_TOKEN = "89b06c10468af620747b4bd340f72fa5d56f6849"
    try:
        wandb.login(key=WANDB_TOKEN)
        os.environ["WANDB_PROJECT"] = "mesh-dit-3d-generation"
        use_wandb = True
        print("Successfully logged in to W&B.")
    except Exception as e:
        print(f"Could not log in to W&B: {e}. Disabling W&B.")
        use_wandb = False


    class CustomDataCollator:
        def __call__(self, features):
            batch = {}
            batch['x'] = torch.stack([f['x'] for f in features])
            y_features = [f['y'] for f in features]
            batch['y'] = {key: torch.stack([d[key] for d in y_features]) for key in y_features[0]}
            return batch
    
    class MeshDiTTrainer(Trainer):
        def __init__(self, *args, noise_scheduler, **kwargs):
            super().__init__(*args, **kwargs)
            self.noise_scheduler = noise_scheduler

        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            x_start = inputs["x"]
            model_kwargs = inputs["y"]
            
            noise = torch.randn_like(x_start)
            batch_size = x_start.shape[0]
            
            # 2. Sample Timesteps
            timesteps = torch.randint(0, self.noise_scheduler.config.num_train_timesteps, 
                                      (batch_size,), device=x_start.device).long()
            noisy_latents = self.noise_scheduler.add_noise(x_start, noise, timesteps)
            noise_pred = model(noisy_latents, timesteps, model_kwargs)
            alphas = self.noise_scheduler.alphas_cumprod.to(x_start.device)
            alpha_t = alphas[timesteps]
            snr = alpha_t / (1 - alpha_t + 1e-8)
            gamma = 5.0
            weights = torch.clamp(snr, max=gamma) / snr
            
            loss_elementwise = F.mse_loss(noise_pred, noise, reduction="none")
            loss_per_sample = loss_elementwise.mean(dim=list(range(1, loss_elementwise.ndim)))
            loss = (loss_per_sample * weights).mean()

            return (loss, {"loss": loss}) if return_outputs else loss

        def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
            inputs = self._prepare_inputs(inputs)
            with torch.no_grad():
                loss = self.compute_loss(model, inputs)
            return (loss, None, None)


    class ConcatenatedMeshDataset(Dataset):
        def __init__(self, pt_paths):
            self.pt_paths = pt_paths
            self.cumsum = [0]
            self.lengths = []
            self.file_handles = []
            for path in pt_paths:
                print(f"Opening {os.path.basename(path)} with mmap (zero copy)...")
                handle = torch.load(path, map_location='cpu', mmap=True)
                data_list = handle['data']
                length = len(data_list)
                self.lengths.append(length)
                self.cumsum.append(self.cumsum[-1] + length)
                self.file_handles.append(data_list)
            print(f"Total samples: {sum(self.lengths):,} across {len(pt_paths)} files")

        def __len__(self):
            return self.cumsum[-1]

        def __getitem__(self, idx):
            for i, cum in enumerate(self.cumsum):
                if idx < cum:
                    file_idx = i - 1
                    local_idx = idx - self.cumsum[file_idx]
                    break
            data_list = self.file_handles[file_idx]
            sample = data_list[local_idx]
            return {'x': sample['x'], 'y': sample['y']}

    class EMACallback(TrainerCallback):
        def __init__(self, decay=0.9999):
            self.decay = decay
            self.shadow = {}

        def on_train_begin(self, args, state, control, model, **kwargs):
            self.shadow = {n: p.clone().detach() for n, p in model.named_parameters() if p.requires_grad}
            print(f"EMA initialized with decay {self.decay}")

        def on_step_end(self, args, state, control, model, **kwargs):
            with torch.no_grad():
                for n, p in model.named_parameters():
                    if n in self.shadow:
                        self.shadow[n] = self.shadow[n].to(p.device) # Ensure device match
                        self.shadow[n] = self.decay * self.shadow[n] + (1 - self.decay) * p.detach()

        def on_save(self, args, state, control, **kwargs):
            ema_path = os.path.join(args.output_dir, f"checkpoint-{state.global_step}", "pytorch_model_ema.bin")
            os.makedirs(os.path.dirname(ema_path), exist_ok=True)
            torch.save(self.shadow, ema_path)
            print(f"EMA weights saved to {ema_path}")

        def on_train_end(self, args, state, control, model, **kwargs):
            print("Loading EMA weights into model for final save...")
            for n, p in model.named_parameters():
                if n in self.shadow:
                    p.data.copy_(self.shadow[n].to(p.device))

    full_dataset = ConcatenatedMeshDataset(local_paths)
    total_len = len(full_dataset)
    eval_size = max(1, int(total_len * 0.10))
    train_size = total_len - eval_size
    train_subset = Subset(full_dataset, range(eval_size, total_len))
    eval_subset = Subset(full_dataset, range(eval_size))

    collator = CustomDataCollator()
    scheduler = DDIMScheduler(num_train_timesteps=2000, beta_schedule="linear", prediction_type="epsilon")

    
    generator = torch.Generator().manual_seed(42)
    train_sampler = RandomSampler(train_subset, generator=torch.Generator().manual_seed(42))
    eval_sampler = SequentialSampler(eval_subset)
    
    print(f"Train: {train_size:,} | Eval: {eval_size:,}")



    print("\n" + "="*6060)
    print("PHASE 1: Training from scratch (FAST!)")
    print("="*60)

    model_p1 = MeshDiT_models['MeshDiT-S'](
        input_tokens=2048,
        vocab_size=tokenizer.vocab_size,
        use_rope=True,
        use_ternary_rope=True,
        image_condition=False,
        full_precision=True,
        optimized_bitlinear=False,
    ).to(device, dtype=dtype)

    model_path = kagglehub.dataset_download("sakibahmed2022/meshdit-trained-model")    
    model_path=os.path.join(model_path, "model.safetensors")
    state_dict_p1 = safetensors_load(model_path, device="cpu")
    model_p1.load_state_dict(state_dict_p1)
    print("✅ Phase 1 weights loaded for fine-tuning")
    

    print(model_p1)
    
    args_p1 = TrainingArguments(
        output_dir="./phase1_ckpt",
        num_train_epochs=10,
        per_device_train_batch_size=64, 
        gradient_accumulation_steps=8,
        
        learning_rate=2e-5,          
        max_grad_norm=1.0,           
        weight_decay=0.02,           
        warmup_ratio=0.05,
        
        lr_scheduler_type="cosine",  
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss", 
        greater_is_better=False,
        fp16=True,
        dataloader_num_workers=4,
        remove_unused_columns=False,
        report_to="wandb" if use_wandb else "tensorboard",
        run_name="MeshDiT-S-Optimized",
        torch_compile=True,        
        torch_compile_backend="inductor",
    )
    
    trainer_p1 = MeshDiTTrainer(
        model=model_p1,
        args=args_p1,
        train_dataset=train_subset,
        eval_dataset=eval_subset,
        data_collator=collator,
        noise_scheduler=scheduler,
    )
    
    trainer_p1.train_dataloader = DataLoader(
        train_subset,
        batch_size=args_p1.per_device_train_batch_size,
        sampler=train_sampler,
        collate_fn=collator,
        num_workers=4,
        pin_memory=True,
        prefetch_factor=4,
        persistent_workers=True
    )
    
    trainer_p1.eval_dataloader = DataLoader(
        eval_subset,
        batch_size=args_p1.per_device_train_batch_size,
        sampler=eval_sampler,
        collate_fn=collator,
        num_workers=4,
        pin_memory=True
    )
    
    trainer_p1.add_callback(EMACallback(decay=0.9999))

    trainer_p1.train()
    
    trainer_p1.save_model("./mesh_dit_final")
    tokenizer.save_pretrained("./mesh_dit_final")
    
    if use_wandb:
        wandb.finish()
        
    print("\nTRAINING COMPLETE!")



In [10]:
if __name__ == "__main__":
    train_model()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to W&B.
Opening all_data.pt with mmap (zero copy)...
Opening all_data_1.pt with mmap (zero copy)...
Opening all_data_2.pt with mmap (zero copy)...
Opening all_data_3.pt with mmap (zero copy)...
Opening all_data_4.pt with mmap (zero copy)...


INFO:mmfreelm.models.hgrn_bit.mesh_dit:Absolute positional embeddings are disabled for this model.


Total samples: 97,308 across 5 files
Train: 87,578 | Eval: 9,730

PHASE 1: Training from scratch (FAST!)
Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.

The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.


...

W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233] WON'T CONVERT torch_dynamo_resume_in_forward_at_136 /workspace/AI-Game-Engine/mmfreelm/layers/hgrn_bit.py line 136 
W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233] due to: 
W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233] Traceback (most recent call last):
W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233]   File "/opt/conda/lib/python3.11/site-packages/torch/_dynamo/convert_frame.py", line 1164, in __call__
W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233]     result = self._inner_convert(
W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233]              ^^^^^^^^^^^^^^^^^^^^
W1127 19:16:01.374000 969 site-packages/torch/_dynamo/convert_frame.py:1233]   File "/opt/conda/lib/python3.11/site-packages/torch/_dynamo/convert_frame.py", line 547, in __call__
W1127 19:16:01.374000 969 

Step,Training Loss,Validation Loss
500,0.102800,0.104388
1000,0.103100,0.104266
1500,0.103500,0.101336


EMA weights saved to ./phase1_ckpt/checkpoint-1000/pytorch_model_ema.bin
EMA weights saved to ./phase1_ckpt/checkpoint-1720/pytorch_model_ema.bin
Loading EMA weights into model for final save...


eval/loss,██▁
eval/runtime,█▇▁
eval/samples_per_second,▁▁█
eval/steps_per_second,▁▁█
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▆█▇▄▄▄▅▅▄▄▇▇▆▆▅▅▆▅▅▆▅▄▃▅▄▄▁▂▂▃▁▁▁▁
train/learning_rate,▅██████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
train/loss,▆▆█▄▆▆▇▅▇▄▃█▅▃▅▃▆▄▅▄▂▂▃▅▅▁▃▅▃▅▆▃▅▃
eval/loss,0.10134
eval/runtime,92.3751



TRAINING COMPLETE!


In [11]:
import os
import json
import shlex
import subprocess
from datetime import datetime

KAGGLE_USERNAME = "sakibahmed2022"
KAGGLE_KEY = "d0ec7fe8091a3906c6995568c11e9e58"
os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
os.environ['KAGGLE_KEY'] = KAGGLE_KEY

DESIRED_SLUG    = f"{KAGGLE_USERNAME}/meshdit-trained-model"
LOCAL_DIR       = "./mesh_dit_final"
TITLE           = "MeshDiT S Trained Model 10 epoch"
VERSION_MSG     = f"Upload final model – {datetime.utcnow().isoformat()} UTC"
PUBLIC          = True          


def write_metadata(folder: str, dataset_id: str):
    meta = {
        "title": TITLE,
        "id": dataset_id,
        "licenses": [{"name": "CC-BY-4.0"}],
        "description": f"{TITLE} – uploaded {datetime.utcnow().isoformat()} UTC"
    }
    path = os.path.join(folder, "dataset-metadata.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)
    print(f"[meta] {path}")


def try_api(folder: str, dataset_id: str, msg: str):
    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()

    try:
        api.dataset_create_version(
            folder=folder,
            version_notes=msg,
            convert_to_csv=False,
            delete_old_versions=False,
        )
        print("[api] version created")
        return True, dataset_id
    except Exception as e:
        if "403" in str(e):
            print("[api] 403 after file upload → will use CLI")
            return False, dataset_id
        # Dataset missing → create new
        if "does not exist" in str(e).lower() or "404" in str(e):
            print("[api] dataset missing → creating new")
            api.dataset_create_new(folder=folder, public=PUBLIC, convert_to_csv=False)
            print("[api] new dataset created")
            return True, dataset_id
        print(f"[api] unexpected error: {e}")
        return False, None


def try_cli(folder: str, dataset_id: str, msg: str):
    # 1. version
    cmd = f"kaggle datasets version -p {shlex.quote(folder)} -m {shlex.quote(msg)}"
    print(f"[cli] version: {cmd}")
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if p.returncode == 0:
        print("[cli] version OK")
        return True, dataset_id

    # 2. create (only if version failed because dataset missing)
    cmd = f"kaggle datasets create -p {shlex.quote(folder)}"
    print(f"[cli] create: {cmd}")
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if p.returncode == 0:
        print("[cli] create OK")
        return True, dataset_id

    print("[cli] both commands failed")
    print("STDOUT:", p.stdout)
    print("STDERR:", p.stderr)
    return False, None


def upload():
    if not os.path.isdir(LOCAL_DIR):
        raise FileNotFoundError(LOCAL_DIR)

    files = [f for f in os.listdir(LOCAL_DIR) if os.path.isfile(os.path.join(LOCAL_DIR, f))]
    print("[main] files:", files)

    # Resolve slug
    dataset_id = DESIRED_SLUG
    ts = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    alt = f"{KAGGLE_USERNAME}/meshdit-trained-model-{ts}"

    # Try a cheap API call to see if the slug exists
    try:
        from kaggle.api.kaggle_api_extended import KaggleApi
        KaggleApi().authenticate()
        KaggleApi().dataset_list_files(dataset_id)   # raises if missing
        print(f"[slug] '{dataset_id}' exists → using timestamped")
        dataset_id = alt
    except Exception:
        pass   # keep desired slug

    write_metadata(LOCAL_DIR, dataset_id)

    # 1. API
    print("[main] trying Python API …")
    ok, final_id = try_api(LOCAL_DIR, dataset_id, VERSION_MSG)
    if ok:
        url = f"https://www.kaggle.com/datasets/{final_id}"
        print(f"[SUCCESS] API → {url}")
        return url

    # 2. CLI
    print("[main] API failed → CLI fallback …")
    ok, final_id = try_cli(LOCAL_DIR, dataset_id, VERSION_MSG)
    if ok:
        url = f"https://www.kaggle.com/datasets/{final_id}"
        print(f"[SUCCESS] CLI → {url}")
        return url

    raise RuntimeError("Both API and CLI failed")


if __name__ == "__main__":
    final_url = upload()
    print("\n=== FINAL URL ===")
    print(final_url)
    print("\nDownload with:")
    print(f"kaggle datasets download -d {final_url.split('/')[-2]}/{final_url.split('/')[-1]}")

[main] files: ['model.safetensors', 'training_args.bin', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.json']
[slug] 'sakibahmed2022/meshdit-trained-model' exists → using timestamped
[meta] ./mesh_dit_final/dataset-metadata.json
[main] trying Python API …
Starting upload for file model.safetensors


100%|██████████| 964M/964M [00:06<00:00, 159MB/s]  


Upload successful: model.safetensors (964MB)
Starting upload for file training_args.bin


100%|██████████| 5.30k/5.30k [00:00<00:00, 13.2kB/s]


Upload successful: training_args.bin (5KB)
Starting upload for file tokenizer_config.json


...

100%|██████████| 437/437 [00:00<00:00, 998B/s]


Upload successful: special_tokens_map.json (437B)
Starting upload for file tokenizer.json


100%|██████████| 3.34M/3.34M [00:00<00:00, 7.42MB/s]


Upload successful: tokenizer.json (3MB)
[api] 403 after file upload → will use CLI
[main] API failed → CLI fallback …
[cli] version: kaggle datasets version -p ./mesh_dit_final -m 'Upload final model – 2025-11-27T22:28:40.542092 UTC'


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 152: ordinal not in range(128)